In [198]:
##Values to adapt
restaurantID = 'harpers_montparnasse'
email = 'afaf.aimenn@gmail.com'
email = 'macquetraphael@gmail.com'
fixeStripe = 0.25
variableStrip = 0.986

import datetime

import os
#os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/Users/raph/Documents/Zeep/App/zeepwebapp-6535a41e25af.json"

from email.mime.multipart import MIMEMultipart
from email.mime.application import MIMEApplication
from email.mime.text import MIMEText
import smtplib
import firebase_admin
from google.cloud import firestore
from datetime import datetime
from datetime import date
import datetime
from datetime import timezone as tz
from datetime import *
import pytz
import json
from exponent_server_sdk import (
    DeviceNotRegisteredError,
    PushClient,
    PushMessage,
    PushServerError,
    PushTicketError,
)
from fpdf import FPDF
from os import path, environ
# Importing from configuration.yaml file
#from config import configuration


from requests.exceptions import ConnectionError, HTTPError
db = firestore.Client()

root = path.dirname(path.abspath(__file__))

print(root)

def sendEmail():
    today = str(datetime.now())
    date = today[8:10]+"/"+today[5:7]+"/"+today[0:4]
    date = '13/07/2022'
    print(today)
    print(date)


    restaurants = db.collection(u'restaurants')
    docs = restaurants.where(u'id', u'==', restaurantID).stream()
    restaurantName = ''
    adress = ''
    category = ''
    for doc in docs:
        restaurantName = u'{}'.format(doc.to_dict()['name'])
        adress = u'{}'.format(doc.to_dict()['adress'])
        category = u'{}'.format(doc.to_dict()['category'])
    
    print(restaurantName)
    print(adress)
    print(category)

    orders = db.collection(u'orders')
    docs = orders.where(u'Date', u'==', date).where(u'restaurantID',u'==', restaurantID).stream()
    nbOrders = 0
    nbPeople = 0
    totalAmount = 0
    for doc in docs:
        if(u'{}'.format(doc.to_dict()['isOriginal'])=="True"):
            print(u'{}'.format(doc.to_dict()['NbOfPeople']))
            nbPeople += int(u'{}'.format(doc.to_dict()['NbOfPeople']))
            #docId = orders.document(doc.id)
        #docId.update({'FirstName':'Raph'})
        #token = u'{}'.format(doc.to_dict()['pushToken'])
        nbOrders += 1
        totalAmount += float(u'{}'.format(doc.to_dict()['TotalPrice']))
    commissionStripe = totalAmount*variableStrip - nbPeople*0.25
    commissionStripe = float(f'{commissionStripe:.2f}')

    print(nbPeople)
    print(nbOrders)
    print(totalAmount)
    print(commissionStripe)

    class PDF(FPDF):
        def header(self):
            # Logo
            self.image('https://zeep-app.com/wp-content/uploads/2021/08/cropped-logoShort.png', 10, 8, 20)
            # Arial bold 15
            self.set_font('Arial', 'B', 15)
            # Move to the right
            self.cell(45)
            # Title
            self.cell(30, 10, "Récapitulatif des commandes du "+date,'C')#, 1, 0, 'C')
            # Line break
            self.ln(20)

            # Page footer
        def footer(self):
            # Position at 1.5 cm from bottom
            self.set_y(-15)
            # Arial italic 8
            #self.set_font('Arial', 'I', 8)
            # Page number
            #self.cell(0, 10, 'Page ' + str(self.page_no()) + '/{nb}', 0, 0, 'C')

    # Instantiation of inherited class
    pdf = PDF()
    pdf.alias_nb_pages()
    pdf.add_page()
    pdf.set_font('Helvetica', '', 15)
    pdf.cell(200, 10, txt = restaurantName,
         ln = 2, align = 'C')

    pdf.cell(200, 10, txt = adress,
         ln = 2, align = 'C')

    pdf.ln(50)

    pdf.set_font('Helvetica', '', 12)

    pdf.cell(200, 10, txt = 'Nombre de clients total : ' + str(nbPeople) + " personnes",
         ln = 2)

    pdf.cell(200, 10, txt = 'Nombre de réservations différentes : ' + str(nbOrders) + " personnes",
         ln = 2)

    pdf.cell(200, 10, txt = 'Montat total avant commission Stripe : ' + str(totalAmount) + " euros",
         ln = 2)

    pdf.cell(200, 10, txt = 'Montat total après commission Stripe : ' + str(commissionStripe) + " euros",
         ln = 2)

    pdf.set_font('Helvetica', '', 10)

    pdf.ln(60)

    pdf.cell(200, 10, txt = 'Nous vous rappelons que les montants qui transitent par Zeep seront versés sur le compte bancaire transmis,',
         ln = 2)
    pdf.cell(200, 10, txt = 'aux dates convenues.',
         ln = 2)
    pdf.ln(10)

    pdf.cell(200, 10, txt = 'Pour toute demande, veillez envoyer un mail à l\'adresse suivante : contact@zeep-app.com',
         ln = 2, )

    pdf.ln(10)

    pdf.cell(200, 10, txt = 'L\'équipe Zeep reste à votre disposition et vous remercie de votre confiance.',
         ln = 2)

    pdf.ln(5)

    pdf.set_font('Helvetica', '', 8)

    pdf.cell(200, 10, txt = 'Zeep SAS, enregistré sous le numéro 901588657 - www.zeep-app.com',
         ln = 2, align = 'C')

    pdf.output('recap.pdf', 'F')

    if(totalAmount!=0):
        msg = MIMEMultipart()
        body_part = MIMEText("""Bonjour, 

Veuillez trouver ci-joint votre récapitulatif quotidien, 

Cordialement,

La team Zeep.
    
        """, 'plain')
        msg['Subject'] = 'Zeep: Récapitulatif du '+date
        msg['From'] = 'finance@zeep-app.com'
        msg['To'] = email
        sender_address ='finance@zeep-app.com' 
        sender_pass = 'ZEEPEDHEC!'
        FILE_NAME = '/dbfs/mnt/mnt1/Inputs/RefTableCoordinates.csv'
        # Add body to email
        msg.attach(body_part)
        # open and read the CSV file in binary
        #with open(pdf) as f:
            #attach = email.mime.application.MIMEApplication(f.read(),_subtype="pdf")
        attach = MIMEApplication(pdf,_subtype="pdf")
        attach.add_header('Content-Disposition','attachment',filename=str('recap-'+date+'.pdf'))
        msg.attach(attach)
        # Create SMTP object
        """smtp_obj = smtplib.SMTP('smtp.gmail.com', 587)
        # Login to the server
        smtp_obj.login(sender_address, sender_pass)
        # Convert the message to a string and send it
        smtp_obj.sendmail(msg['From'], msg['To'], msg.as_string())
        smtp_obj.quit()"""
        
        session = smtplib.SMTP('ssl0.ovh.net', 587) 
        session.starttls() #enable security
        session.login(sender_address, sender_pass) #login with mail_id and password
        #text = message.as_string()
        session.sendmail(msg['From'], msg['To'], msg.as_string())
        session.quit()
        print('Mail Sent')

sendEmail()

NameError: name '__file__' is not defined

In [196]:
!pip install config

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621

[notice] A new release of pip available: 22.1.2 -> 22.2.1
[notice] To update, run: python3.9 -m pip install --upgrade pip
